---
title: pyOpenSci Software Submission Stats
subtitle: pyOpenSci Peer Review Summary Stats
author:
  - name: Leah Wasser
    affiliations: pyOpenSci
    orcid: 0000-0002-7859-8394
    email: leah@pyopensci.org
license:
  code: MIT
date: 2024/06/19
---


* https://github.com/ryantam626/jupyterlab_code_formatter

This is a workflow that colates all GitHub issues associated with our reviews. 

Questions i have

* How to add figure captions and alt text
* How to create this same thing but using a markdown file that runes code so i don't have to commit notebooks.
* 

In [1]:
import os
from datetime import datetime

import altair as alt
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from pyosmeta import ProcessIssues
from pyosmeta.github_api import GitHubAPI

In [2]:
def parse_single_issue(issue) -> dict:
    """
    Parse a single issue from the GitHub API response.

    Parameters
    ----------
    issue : dict
        Dictionary containing information about a single issue.

    Returns
    -------
    dict
        Dictionary containing parsed information about the issue.
    """
    parsed_issue = {}

    # Extract labels
    parsed_issue["labels"] = [label["name"] for label in issue.get("labels", [])]

    # Extract header text (title of the issue)
    parsed_issue["header_text"] = issue.get("title", "")

    # Extract date opened
    parsed_issue["date_opened"] = datetime.strptime(
        issue.get("created_at"), "%Y-%m-%dT%H:%M:%SZ"
    )

    # Extract date closed (if available)
    if issue.get("closed_at"):
        parsed_issue["date_closed"] = datetime.strptime(
            issue.get("closed_at"), "%Y-%m-%dT%H:%M:%SZ"
        )
        # Calculate total time issue was open
        time_open = parsed_issue["date_closed"] - parsed_issue["date_opened"]
        parsed_issue["time_open_days"] = time_open.total_seconds() / (60 * 60 * 24)
    else:
        parsed_issue["date_closed"] = None
        parsed_issue["time_open_days"] = None

    return parsed_issue

In [3]:
# Get all issues from GitHub software-submission repo, Return df with labels, title, date_opened and closed and total time open in days
github_api = GitHubAPI(
    org="pyopensci",
    repo="software-submission",
)

process_review = ProcessIssues(github_api)
issues = process_review.return_response()

all_issues = []
for issue in issues:
    all_issues.append(parse_single_issue(issue))

df = pd.DataFrame(all_issues)

# Remove issues that are unlabeled or say help wanted
valid_issues = df[
    ~(
        (df["labels"].apply(len) == 0)
        | df["labels"].apply(lambda x: "help wanted" in x or "Help Request" in x)
    )
]

# Total presubmissions - get the total number of pre-submission inquiries (all time)
total_presubmissions = valid_issues[
    valid_issues["labels"].apply(lambda x: "presubmission" in x)
].shape[0]

# Total Presubmissions

Here we removed all issues that were help-wanted or issus with our templates that were not related to a software-review submission. As of today we have had **{eval}`total_presubmissions` presubmission inquiries** submitted to pyOpenSci.

## Summary - Number of Issues by year

In [4]:
# Group issues by year and get counts
annual_issues = valid_issues.copy()

# Create a new column 'year' by extracting the year from the 'date_opened' column
annual_issues.loc[:, "year"] = annual_issues["date_opened"].dt.year

In [5]:
# Add year / month
annual_issues["year_month"] = annual_issues["date_opened"].dt.to_period("M")
counts_month_year = annual_issues.groupby("year_month").size().reset_index(name="count")

In [6]:
# Create a complete range of year_month periods
# Note i use this below - don't have to recalculate
all_month_years = pd.period_range(
    start=counts_month_year["year_month"].min(),
    end=counts_month_year["year_month"].max(),
    freq="M",
)

In [7]:
issues_by_year = (
    annual_issues.groupby("year")
    .size()
    .reset_index(name="count")
    .sort_values(by="year", ascending=False)
    .reset_index(drop=True)
)

In [8]:
# Create an Altair bar chart
chart = (
    alt.Chart(issues_by_year)
    .mark_bar(color="purple")
    .encode(
        x=alt.X(
            "year:O", axis=alt.Axis(labelAngle=0, labelFontSize=14, titleFontSize=18),
            sort=alt.EncodingSortField(field="year", order="descending"),
        ),
        y=alt.Y(
            "count:Q",
            axis=alt.Axis(labelFontSize=14, titleFontSize=18),
            
        ),
        tooltip=["year", "count"],
    )
    .properties(
        title=alt.TitleParams(
            text="pyOpenSci -- Number of Issues by Year", fontSize=24
        ),
        width=600,
    )
)

chart.show()

alt.Chart(...)

In [9]:
# Get fill in months with no issues with a value of 0
month_year_counts = (
    counts_month_year.set_index("year_month")
    .reindex(all_month_years, fill_value=0)
    .rename_axis("year_month")
    .reset_index()
)

# Summary: issues by month / year

Below you can see scientific Python peer review issues submitted by month since 2019. 

In [10]:
# Split year_month into separate year and month columns
month_year_counts["year"] = month_year_counts["year_month"].dt.year
month_year_counts["month"] = month_year_counts["year_month"].dt.strftime("%b")
month_year_counts["month_cat"] = pd.Categorical(
    month_year_counts["month"],
    categories=[
        "Jan",
        "Feb",
        "Mar",
        "Apr",
        "May",
        "Jun",
        "Jul",
        "Aug",
        "Sep",
        "Oct",
        "Nov",
        "Dec",
    ],
    ordered=True,
)
month_year_counts = month_year_counts.drop(columns=["year_month"])

## Monthly patterns

which months are most popular?

In [18]:
# Create an Altair bar plot with a grid layout
chart = (
    alt.Chart(month_year_counts)
    .mark_bar()
    .encode(
        x=alt.X(
            "month:N",
            title="Month",
            sort=[
                "Jan",
                "Feb",
                "Mar",
                "Apr",
                "May",
                "Jun",
                "Jul",
                "Aug",
                "Sep",
                "Oct",
                "Nov",
                "Dec",
            ],
        ),
        y=alt.Y("count:Q", title="Number of Issues"),
        color=alt.Color("year:N", scale=alt.Scale(scheme="bluepurple")),
    )
    .properties(width=200, height=300, title="Number of Issues Per Month and Year")
    .facet(
        facet=alt.Facet(
            "year:N",
            title="Year",
            sort="descending",
            header=alt.Header(labelAngle=0),
        ),
        columns=3,
    )
)

chart.show()

alt.FacetChart(...)

## Monthly Trends

Things seem to slow down in July and then again around the holidays --  November / December.

In [22]:
# Create an Altair bar plot with a grid layout where each facet represents a month
chart = (
    alt.Chart(month_year_counts)
    .mark_bar()
    .encode(
        x=alt.X("year:N", title="Year", axis=alt.Axis(labelAngle=0)),
        y=alt.Y("count:Q", title="Number of Issues"),
        color=alt.Color("year:N", scale=alt.Scale(scheme="bluepurple")),
        tooltip=[
            alt.Tooltip("year:N", title="Year"),
            alt.Tooltip("month:N", title="Month"),
            alt.Tooltip("count:Q", title="Count"),
        ],
    )
    .properties(width=150, height=300, title="Number of Issues Per Month and Year")
    .facet(
        facet=alt.Facet(
            "month:N",
            title="Month",
            sort=[
                "Jan",
                "Feb",
                "Mar",
                "Apr",
                "May",
                "Jun",
                "Jul",
                "Aug",
                "Sep",
                "Oct",
                "Nov",
                "Dec",
            ],
            header=alt.Header(labelAngle=0,labelFontSize=20, labelFontWeight='bold'),
        ),
        columns=4,
    )
    .resolve_scale(x="independent")  # Ensure that each facet has its own x-axis scale
)

chart.show()

alt.FacetChart(...)

# Issues opened by month / year

# Number of Issues per Month Since 2019

Below is a cumulative sum representation of all of our peer review issues submitted to date. You can see that there is a significant uptick of issues submitted that began when we were able to utilize our funding and have a full time staff person (the Executive Director) onboard. 

In [23]:
# Set 'date_opened' column as index / add month and year cols for grouping
monthly_issues = valid_issues.copy()
monthly_issues["month"] = monthly_issues["date_opened"].dt.month
monthly_issues["year"] = monthly_issues["date_opened"].dt.year
# Get monthly counts
monthly_issues_index = monthly_issues.copy()

monthly_issues_index.set_index(
    monthly_issues_index["date_opened"].dt.to_period("M").dt.strftime("%Y-%m"),
    inplace=True,
)

# Group by the new index (month-year) and count the number of issues for each month-year
monthly_counts = monthly_issues_index.groupby(level=0).size()

In [14]:
# Create a df for every month/year combo in our dataset - this ensures a date for every
# month even if some months are missing
all_month_years = pd.date_range(
    start=monthly_issues.date_opened.min().strftime("%Y-%m"),
    end=monthly_issues.date_opened.max().strftime("%Y-%m"),
    freq="MS",
).to_period("M")

In [24]:
final_monthly = monthly_counts.copy()
# Ensure the index is of type periodIndex to support reindexing
final_monthly.index = pd.PeriodIndex(final_monthly.index, freq="M")
final_monthly = final_monthly.reindex(all_month_years, fill_value=0).to_frame(
    name="issue_count"
)

# Calculate cumulative sum of issue count
final_monthly["cumulative_count"] = final_monthly["issue_count"].cumsum()
final_monthly.reset_index(inplace=True, names="date")
final_monthly["date"] = final_monthly["date"].dt.to_timestamp()

In [26]:
# Create an Altair line plot
chart = (
    alt.Chart(final_monthly)
    .mark_line(color="purple", strokeWidth=8)
    .encode(
        x=alt.X(
            "date:T",
            axis=alt.Axis(
                title="Month",
                titleFontSize=18,
                labelFontSize=14,
                format="%b-%Y",
                tickCount="year",
            ),
        ),
        y=alt.Y(
            "cumulative_count:Q",
            axis=alt.Axis(
                title="Number of Issues",
                titleFontSize=18,
                labelFontSize=14,
                tickMinStep=50,
            ),
        ),
        tooltip=[
            alt.Tooltip("date:T", title="Month"),
            alt.Tooltip("cumulative_count:Q", title="Number of Issues"),
        ],
    )
    .properties(
        title=alt.TitleParams(
            text="pyOpenSci Software Review Issues: Cumulative Issues Over Time",
            fontSize=25,
            anchor="start",
            offset=20,
        ),
        width=600,
        height=400,
    )
)

# Show the chart
chart.show()

alt.Chart(...)